In [192]:
import pandas as pd
import pyspark
import os
import folium

from pyspark.sql import *
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import *

In [193]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [194]:
txt_file = sc.textFile('../data/GdeltDomainsByCountry-May18.txt')

In [205]:
rdd = txt_file.map(lambda x: x.split("\t"))
df = rdd.toDF(['web', 'code', 'country'])

df_sum = df.groupBy(['country', 'code']).count()
df_sum = df_sum.sort(['count'], ascending=False)
df_sum_pd = df_sum.toPandas()

In [206]:
df_sum_pd.head(10)

,country,code,count
0,United States,US,56160
1,United Kingdom,UK,14892
2,Russia,RS,8216
3,Italy,IT,7672
4,Canada,CA,7331
5,Germany,GM,6085
6,Australia,AS,6022
7,India,IN,4453
8,France,FR,4138
9,Brazil,BR,3814


In [202]:
country_info = pd.read_csv('../data/countries-info.csv')
country_info = country_info[['name', 'alpha-2', 'alpha-3', 'region', 'sub-region']]
country_info.head(5)

,name,alpha-2,alpha-3,region,sub-region
0,Afghanistan,AF,AFG,Asia,Southern Asia
1,Åland Islands,AX,ALA,Europe,Northern Europe
2,Albania,AL,ALB,Europe,Southern Europe
3,Algeria,DZ,DZA,Africa,Northern Africa
4,American Samoa,AS,ASM,Oceania,Polynesia


In [211]:
len(df_sum_pd)

241

In [197]:
df_m = pd.merge(country_info, df_sum_pd, left_on='alpha-2', right_on='code')
df_m.sort_values(['count'], ascending=False).head(20)

,name,alpha-2,alpha-3,region,sub-region,country,code,count
156,United States of America,US,USA,Americas,Northern America,United States,US,56160
133,Serbia,RS,SRB,Europe,Southern Europe,Russia,RS,8216
71,Italy,IT,ITA,Europe,Southern Europe,Italy,IT,7672
26,Canada,CA,CAN,Americas,Northern America,Canada,CA,7331
54,Gambia,GM,GMB,Africa,Sub-Saharan Africa,Germany,GM,6085
2,American Samoa,AS,ASM,Oceania,Polynesia,Australia,AS,6022
67,India,IN,IND,Asia,Southern Asia,India,IN,4453
52,France,FR,FRA,Europe,Western Europe,France,FR,4138
19,Brazil,BR,BRA,Americas,Latin America and the Caribbean,Brazil,BR,3814
103,Netherlands,NL,NLD,Europe,Western Europe,Netherlands,NL,3525


In [199]:
world_map = '../data/countries-land-10km.geo.json'
m = folium.Map(location=[46.8, 8.3], tiles='cartodbpositron', zoom_start=2)
m.choropleth(geo_data=world_map, data=pltdata,
             columns=['alpha-3', 'count'],
             key_on='feature.properties.A3',
             fill_color='YlGnBu', fill_opacity=0.6, line_opacity=0.2)
m
